In [1]:
# Importing the libary
from twelvedata import TDClient
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import glob



In [2]:
# Importing path from src
import sys
import yaml

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))

from src.utils.config import RAW_DATA_DIR, PROCESSED_DATA_DIR, BASE_DIR, RAW_DATA_STOCK_DIR
from src.utils.log_handler import logger
from src.utils.exception import StockPulseException

with open(os.path.join(BASE_DIR, "config/stock_config.yaml"), "r") as file:
    SETTINGS = yaml.safe_load(file)
logger.info("Succesfully imported the file")

In [3]:
# Loading the dotenv file
load_dotenv()

True

In [4]:
# Laading the api key fron environment variable
API_KEY = os.getenv("TWELVE_DATA_API_KEY")

In [5]:
# Datetime variiable
interval_time = SETTINGS['data']['interval']
output_size = SETTINGS['data']['output_size']
symb = SETTINGS['data']['n_symbol']
day = SETTINGS["data"]["days"]

td = TDClient(API_KEY)

end_date= pd.to_datetime(datetime.now().strftime("%Y-%m-%d"))
start_date = end_date- timedelta(days=day)

td = TDClient(API_KEY)

# Fetching data
obj = td.time_series(
    symbol = symb,
    interval=interval_time,
    start_date = start_date,
    end_date= end_date,
    outputsize = output_size
)

df = obj.as_pandas()

In [6]:
# Sorting the dataset correctly (Mon ---> fri)
df = df.sort_index(ascending=True)


In [7]:
# Having the look at top 5 rows of dataset
df.head()

,open,high,low,close,volume
datetime,,,,,
2025-08-22 14:55:00,1202.33997,1202.48584,1202.04175,1202.09998,4232
2025-08-22 14:56:00,1201.79004,1201.79004,1201.79004,1201.79004,461
2025-08-22 14:57:00,1202.00000,1202.60986,1202.00000,1202.60986,6330
2025-08-22 14:58:00,1202.25000,1202.32500,1202.16000,1202.30000,220
2025-08-22 14:59:00,1202.27000,1202.27500,1202.27000,1202.27500,12


In [8]:
#Having look at the bottom 5 rows of dataset
df.tail()

,open,high,low,close,volume
datetime,,,,,
2025-09-05 15:55:00,1243.19995,1244.02246,1243.03003,1243.04004,8448
2025-09-05 15:56:00,1243.27002,1243.45996,1242.85999,1243.09497,15134
2025-09-05 15:57:00,1242.84998,1243.75000,1242.65002,1243.75000,32626
2025-09-05 15:58:00,1243.37000,1243.63000,1242.92249,1243.23999,26312
2025-09-05 15:59:00,1243.31494,1244.17004,1242.85498,1244.17004,52242


In [9]:
# Saving the loaded data to local 

df.to_csv(os.path.join(RAW_DATA_STOCK_DIR, f"{datetime.now().strftime('%Y-%m-%d')}_raw_data.csv"), mode='w')

keep_file = SETTINGS['data']['keep_days']
for file in glob.glob(os.path.join(RAW_DATA_DIR, "*.csv")):
    file_time= datetime.fromtimestamp(os.path.getmtime(file))
    if (datetime.now() - file_time).days >keep_file:
        os.remove(file)
        print(f"Deleted old file: {file}")